In [1]:
import datetime
import time
import requests
import pandas as pd
from functions import util
import hopsworks
import json
import os
import warnings
warnings.filterwarnings("ignore")

In [2]:
with open('./data/hopsworks-api-key.txt', 'r') as file:
     os.environ["HOPSWORKS_API_KEY"] = file.read().rstrip()
project = hopsworks.login()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1157273


In [3]:
fs = project.get_feature_store() 
secrets = util.secrets_api(project.name)

Connected. Call `.close()` to terminate connection gracefully.
Connected. Call `.close()` to terminate connection gracefully.


In [4]:
AQI_API_KEY = secrets.get_secret("AQI_API_KEY").value
location_str = secrets.get_secret("SENSOR_LOCATION_JSON").value
location = json.loads(location_str)

country=location['country']
city=location['city']
street=location['street']
aqicn_url=location['aqicn_url']
latitude=location['latitude']
longitude=location['longitude']

today = datetime.date.today()

location_str

'{"country": "china", "city": "guangzhou", "street": "tiyuxi", "aqicn_url": "https://api.waqi.info/feed/@8318", "latitude": 23.1291, "longitude": 113.2644}'

In [15]:
yesterday = today - datetime.timedelta(days=1)
day_before_yesterday = today - datetime.timedelta(days=2)

In [5]:
# Retrieve feature groups
air_quality_fg = fs.get_feature_group(
    name='air_quality',
    version=1,
)
weather_fg = fs.get_feature_group(
    name='weather',
    version=1,
)

In [21]:
import requests
import pandas as pd

aq_today_df = util.get_pm25(aqicn_url, country, city, street, today, AQI_API_KEY)
aq_yesterday_df = util.get_pm25(aqicn_url, country, city, street, yesterday, AQI_API_KEY)
aq_day_before_yesterday_df = util.get_pm25(aqicn_url, country, city, street, day_before_yesterday, AQI_API_KEY)


In [22]:
aq_today_df

,pm25,country,city,street,date,url
0,95.0,china,guangzhou,tiyuxi,2024-11-12,https://api.waqi.info/feed/@8318


In [23]:
aq_yesterday_df


,pm25,country,city,street,date,url
0,95.0,china,guangzhou,tiyuxi,2024-11-11,https://api.waqi.info/feed/@8318


In [24]:
aq_day_before_yesterday_df

,pm25,country,city,street,date,url
0,95.0,china,guangzhou,tiyuxi,2024-11-10,https://api.waqi.info/feed/@8318


In [33]:
aq_today_df['pm25_rolling_avg'] = ((aq_today_df["pm25"] + aq_yesterday_df["pm25"] + aq_day_before_yesterday_df["pm25"]) / 3).astype("double")

In [34]:
aq_today_df

,pm25,country,city,street,date,url,pm25_rolling_avg
0,95.0,china,guangzhou,tiyuxi,2024-11-12,https://api.waqi.info/feed/@8318,95.0


In [35]:
aq_today_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   pm25              1 non-null      float32       
 1   country           1 non-null      object        
 2   city              1 non-null      object        
 3   street            1 non-null      object        
 4   date              1 non-null      datetime64[ns]
 5   url               1 non-null      object        
 6   pm25_rolling_avg  1 non-null      float64       
dtypes: datetime64[ns](1), float32(1), float64(1), object(4)
memory usage: 180.0+ bytes


In [30]:
hourly_df = util.get_hourly_weather_forecast(city, latitude, longitude)
hourly_df = hourly_df.set_index('date')

# We will only make 1 daily prediction, so we will replace the hourly forecasts with a single daily forecast
# We only want the daily weather data, so only get weather at 12:00
daily_df = hourly_df.between_time('11:59', '12:01')
daily_df = daily_df.reset_index()
daily_df['date'] = pd.to_datetime(daily_df['date']).dt.date
daily_df['date'] = pd.to_datetime(daily_df['date'])
daily_df['city'] = city
daily_df

Coordinates 23.25°N 113.25°E
Elevation 17.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,city
0,2024-11-12,25.350000,0.0,6.519877,353.659912,guangzhou
1,2024-11-13,23.900000,4.4,11.503113,20.136398,guangzhou
2,2024-11-14,23.799999,1.1,4.553679,341.564941,guangzhou
3,2024-11-15,23.950001,0.2,6.287130,66.370613,guangzhou
4,2024-11-16,24.600000,0.0,6.409617,51.842735,guangzhou
5,2024-11-17,25.450001,0.0,16.375053,33.340725,guangzhou
6,2024-11-18,22.450001,0.0,19.319628,26.564985,guangzhou
7,2024-11-19,21.500000,0.0,17.317459,20.695532,guangzhou
8,2024-11-20,18.650000,0.1,5.351785,19.653913,guangzhou
9,2024-11-21,17.000000,0.0,18.089775,5.710507,guangzhou


In [31]:
daily_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   date                         10 non-null     datetime64[ns]
 1   temperature_2m_mean          10 non-null     float32       
 2   precipitation_sum            10 non-null     float32       
 3   wind_speed_10m_max           10 non-null     float32       
 4   wind_direction_10m_dominant  10 non-null     float32       
 5   city                         10 non-null     object        
dtypes: datetime64[ns](1), float32(4), object(1)
memory usage: 448.0+ bytes


In [36]:
# Insert new data
air_quality_fg.insert(aq_today_df)

2024-11-12 23:34:02,842 INFO: 	1 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1157273/fs/1147976/fg/1345961


Uploading Dataframe: 0.00% |          | Rows 0/1 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: air_quality_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/1157273/jobs/named/air_quality_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x7f60b1862d60>,
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "pm25",
           "min_value": -0.1,
           "max_value": 500.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 669754
         }
       },
       "result": {
         "observed_value": 95.0,
         "element_count": 1,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2024-11-12T03:34:02.000842Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     }
   ],
   "evaluation_parameters": {},
   "statistics": {
     "evaluated_expectations": 1,
     "successful_expectations": 1,
     "uns

In [37]:
# Insert new data
weather_fg.insert(daily_df)

2024-11-12 23:34:24,054 INFO: 	2 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1157273/fs/1147976/fg/1345953


Uploading Dataframe: 0.00% |          | Rows 0/10 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: weather_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/1157273/jobs/named/weather_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x7f60b97c5ee0>,
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "wind_speed_10m_max",
           "min_value": -0.1,
           "max_value": 1000.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 669747
         }
       },
       "result": {
         "observed_value": 4.553679466247559,
         "element_count": 10,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2024-11-12T03:34:24.000054Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column